## Project 3

In [25]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [26]:
df_raw = pd.read_csv("/Users/ianmc213/Downloads/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

### 1. Create Frequency Tables for Variables

In [27]:
# frequency table for prestige and whether or not someone was admitted

In [28]:
pd.crosstab(df.admit, df.prestige, margins=True)

prestige,1.0,2.0,3.0,4.0,All
admit,,,,,
0,28,95,93,55,271
1,33,53,28,12,126
All,61,148,121,67,397


## Part 2. Return of Dummy Variable

### 2.1 Create class or dummy variables for prestige

In [29]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')
print dummy_ranks.head()

   prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0             0             0             1             0
1             0             0             1             0
2             1             0             0             0
3             0             0             0             1
4             0             0             0             1


In [30]:
df = df.join(pd.get_dummies(df['prestige'],prefix="prestige"))

In [31]:
df.head()

,admit,gre,gpa,prestige,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,3.0,0,0,1,0
1,1,660.0,3.67,3.0,0,0,1,0
2,1,800.0,4.00,1.0,1,0,0,0
3,1,640.0,3.19,4.0,0,0,0,1
4,0,520.0,2.93,4.0,0,0,0,1


#### 2.2 When modeling class variables, how many do we need?

Answer: At least 2 class variables are needed.

## Part 3. Hand calculating Odds Ratios

In [33]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [ ]:
# crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

In [34]:
pd.crosstab(df.admit, df.prestige, margins=True)

prestige,1.0,2.0,3.0,4.0,All
admit,,,,,
0,28,95,93,55,271
1,33,53,28,12,126
All,61,148,121,67,397


In [36]:
pd.crosstab(df.admit, df.prestige).apply(lambda r: r/len(df), axis=1)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,0.070529,0.239295,0.234257,0.138539
1,0.083123,0.133501,0.070529,0.030227


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [37]:
from __future__ import division

In [39]:
33 / 61 #54%

0.5409836065573771

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [41]:
93 / 243 #38%

0.38271604938271603

#### 3.3 Calculate the Odds Ratio

In [42]:
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


#### 3.4 Write the finding in a sentence...

Answer:

#### 3.5 Print the crosstab for prestige_4

In [46]:
print pd.crosstab(df['admit'], df['prestige_4.0'], rownames=['admit'])

prestige_4.0    0   1
admit                
0             216  55
1             114  12


#### 3.6 Calculate the OR

In [48]:
12 / 67 #17%

0.1791044776119403

#### 3.7 Write this finding in a sentence...

Answer: 17% of students who applied from a Prestige_4 school were admitted.

## Part 4. Analysis

In [49]:
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


In [50]:
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [51]:
train_cols = data.columns[1:]

In [52]:
logit = sm.Logit(data['admit'], data[train_cols])

#### 4.2 Fit the model

In [53]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [54]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Fri, 25 Aug 2017   Pseudo R-squ.:                 0.08166
Time:                        00:02:08   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.14

In [55]:
print result.conf_int()

                     0         1
gre           0.000074  0.004362
gpa           0.127619  1.431056
prestige_2.0 -1.301337 -0.058936
prestige_3.0 -2.014579 -0.662776
prestige_4.0 -2.371624 -0.735197
intercept    -6.116077 -1.637631


In [56]:
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

In [57]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the Odds Ratio of Prestige_2

Answer: I would interpret that a student coming from a Prestige_2 college (OR of .50), has nearly a 25% better chance than a student from Prestige_3 and nearly a 30% better chance than a Prestige_4 student.

#### 4.6 Interpret the Odds Ratio of GPA

Answer: With an Odds Ratio of 2.18, the highest of the 5 variables...I would interpret that as GPA being heavily weighted when it comes to admission.

## Part 5: Predicted probablities

As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.


In [58]:
from sklearn.utils.extmath import cartesian

In [59]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [72]:
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]


In [73]:
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas

[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [81]:
#enumerating the dataset
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]])) #cannot figure out how to get this to work...I know I'm close, damn

TypeError: slice indices must be integers or None or have an __index__ method

#### 5.1 Recreate the dummy variables

In [79]:
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print data.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


#### 5.2 Make predictions on the enumerated dataset

In [64]:
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])

NameError: name 'combos' is not defined

#### 5.3 Interpret findings of the last 4 observations

Answer: 